In [2]:
# -*- coding: utf-8 -*-
"""
@author: Manuballester

Script for checking all created fields in JIRA. It prints fields from different projects so you'll need to ensure which ones
are in yours.

"""

import os
from jira import JIRA
from jira.client import ResultList
from jira.resources import Issue
from atlassian import Confluence
import pandas as pd
import csv

#Do not put your credentials here. At the moment use env variables, but when we have the secret vault, use it and please purge old tokens.
user = os.environ['JiraUser'] 
apikey = os.environ['JiraToken2']
server = 'https://iceyedev.atlassian.net/'

#Note, when using user specific token, the same token is for all your instances. Not just test or production. Act accordingly.
options = {
 'server': server
}

jira = JIRA(options, basic_auth=(user,apikey))

#and only 3 important lines :D
allfields = jira.fields()
nameMap = {field['name']:field['id'] for field in allfields}
print(nameMap)

{'Parent': 'parent', 'Solution Lifecycle': 'customfield_10990', 'Recommendation': 'customfield_10870', 'Status Category': 'statusCategory', 'Budget Estimation': 'customfield_10991', 'Resolution': 'resolution', 'Approver 1 (Team Lead)': 'customfield_10750', 'Value': 'customfield_10992', 'Source': 'customfield_10630', 'Approver 2 (VP or CFO/CSO/CEO))': 'customfield_10751', 'Preferred Arrival Time': 'customfield_10993', 'Pending reason': 'customfield_10631', 'Company Name': 'customfield_10994', 'HR Approved': 'customfield_10752', 'Flood Extent Delivery': 'customfield_10863', 'Impact': 'customfield_10621', 'Country and city of departure': 'customfield_10742', 'Organizations': 'customfield_10500', 'Primary Root Cause': 'customfield_10984', 'Change type': 'customfield_10622', 'Date start': 'customfield_10864', 'Cleared Response to the Customer': 'customfield_10985', 'Country and city of destination': 'customfield_10743', 'Change risk': 'customfield_10623', 'Date end': 'customfield_10865', 'I

In [ ]:
# -*- coding: utf-8 -*-
"""
@author: Manuballester

Read the docs will guide you.
https://jira.readthedocs.io/en/master/

This script will create a Confluence page containing the As Built Configuration of the satellite. Here are included the 
Assemblies and the components contained in them.

"""

import os
from jira import JIRA
from jira.client import ResultList
from jira.resources import Issue
from atlassian import Confluence
import pandas as pd
import csv

#Do not put your credentials here. At the moment use env variables, but when we have the secret vault, use it and please purge old tokens.
user = os.environ['JiraUser'] 
apikey = os.environ['JiraToken2']
server = 'https://iceyedev.atlassian.net/'

#Note, when using user specific token, the same token is for all your instances. Not just test or production. Act accordingly.
options = {
 'server': server
}

jira = JIRA(options, basic_auth=(user,apikey))

#Modify here the name of the satellite which you want to obtain the As Built Configuration: 
name = 'P211'

search = '"Epic Link" = "ICEYE-' + str(name) + '" and summary ~ "assembly" ORDER BY summary'
issues_in_proj = jira.search_issues(search, startAt=0, maxResults=90)   

ABCL = ""
comps = ""
lista = ""

csvname = str(name) + '_ABCL.csv'

# Write in CSV 
with open(csvname, 'w', newline='') as csvfile:
    fieldnames = ['Assembly', 'Components']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for i in issues_in_proj:
        issue = jira.issue(i)
        ABCL = str(issue.permalink()) + "\n" # str(issue.fields.summary) + ": " +

        searcheck = ' issue in linkedIssues(' + str(issue) + ', "is blocked by") '
        issuescheck = jira.search_issues(searcheck)

        if issuescheck != []:
            continue
        search2 = ' issue in linkedIssues(' + str(issue) + ', "blocks") '
        issues_in_proj2 = jira.search_issues(search2)  

        for j in issues_in_proj2:
            issue2 = jira.issue(j)
            summ = issue2.fields.summary
            nn = issue2.permalink()

            search3 = ' issue in linkedIssues(' + str(issue2) + ', "blocks") '
            issues_in_proj3 = jira.search_issues(search3) 
            if issues_in_proj3 == []:
                comp = str(nn)
                comp = ""
            else:
                comp = '\n' + str(nn) 
                comps = comps + comp + "\n"
                comp = ""

            for k in issues_in_proj3:
                issue3 = jira.issue(k)
                summ = issue3.fields.summary
                nn = issue3.permalink()
                comp = str(nn)
                comps = comps + comp + "\n"
                comp = ""

        writer.writerow({'Assembly': ABCL, 'Components': comps})
        lista = lista + ABCL + comps + "\n\n"
        comps = ""

print(lista)

# confluence.create_page(space="SMS", title="ABCL X12", body='', parent_id=None, type='page', representation='storage', editor='v2')

#Update the Confluence page with all the components information:
# confluence = Confluence(url = server,
#     username = user,
#     password = apikey,
#     cloud = True)

# idpage = confluence.get_page_id(space="SMS", title="ABCL X12")
# status = confluence.update_page(
#     parent_id = None,
#     page_id = idpage,
#     title = "ABCL X12",
#     body = lista,
# )

# #Convert to EXCEL:
# read_file = pd.read_csv (r csvname)
# read_file.to_excel (r'testt.xlsx', index = None, header=True)

# #Word: (links didn't work)
# third_para = mydoc.add_paragraph("This is the third paragraph.\n")
# third_para.add_run(" this is a section at the end of third paragraph")
# mydoc.save("P211_ABCL.docx")

# #Update the Confluence page with all the components information:
# confluence = Confluence(url = server,
#     username = user,
#     password = apikey,
#     cloud = True)

# idpage = confluence.get_page_id(space="SMS", title="ABCL X11")
# status = confluence.update_page(
#     parent_id=None,
#     page_id=idpage,
#     title="ABCL X11",
#     body=content,
# )

# #TXT file: (links didn't work)
# f = open("P211_ABCL.txt","w+")
# f.write(lista)
# f.close()
# m = open('P211_ABCL.txt', 'r')
# content = m.read()
# m.close()

#Do not put your credentials here. At the moment use env variables, but when we have the secret vault, use it and please purge old tokens.
# user = os.environ['JiraUser'] 
# apikey = os.environ['JiraToken2']
# server = 'https://iceyedev.atlassian.net/'

# confluence = Confluence(url = server,
#     username = user,
#     password = apikey,
#     cloud = True)

# confluence.create_page(space="SMS", title="ABCL X18", body='hola', parent_id=None, type='page', representation='storage', editor='v2')
# idpage = confluence.get_page_id(space="SMS", title="ABCL X11")
# status = confluence.update_page(
#     parent_id = None,
#     page_id = idpage,
#     title = "ABCL X11",
#     body = "https://iceyedev.atlassian.net/browse/X11-95", # = lista
# )